# Train 2.0

In [1]:
import argparse
import json
import random
import re
import os
import sys
import keras.backend as K
import theano.tensor as T
import numpy as np
import pickle

from collections import Counter
from keras.layers import Input, Embedding, LSTM, Dense, RepeatVector, Bidirectional, Dropout, merge
from keras.layers.merge import concatenate
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Activation, Dense
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from utils.data_utils import get_data
import configuration as config

Using TensorFlow backend.


## Convolutional model

In [2]:
conv_model = load_model('inception_log2.0/fine_inception.h5')      
conv_model.layers.pop()
conv_model.summary()
#conv_model.layers.pop()
#conv_model.layers.pop()

OSError: Unable to open file (Unable to open file: name = 'inception_log2.0/fine_inception.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

## Text pre-processing

In [ ]:
# sentence markers
BOS = 2
EOS = 3
UNK = 4

r_word = re.compile('(\W+)?')

In [ ]:
# Convierte palabras a indices
def tokenize(sent):
    return [x.strip().lower() for x in r_word.split(sent) if x.strip()]

In [ ]:
dataset_dir = '10k/part-0-to-1000/'
vocab_file = '10k/word_count.txt'
dataset_proportion = 1
images_captions, vocab_size = get_data(dataset_dir, conv_model, vocab_file,
                                       quantity=dataset_proportion)
vocab_size += 1

# Recurrent Neural Model

In [ ]:
# placeholders
maxlen_input = np.shape(images_captions)[0]
caption = Input(shape=(config.embedding_size,), dtype='float32', name='caption')
input_context = Input(shape=(maxlen_input,), dtype='float32', name='input_context')
input_context = Dense(128, name='resize-image')(input_context)

In [ ]:
# embedding layer
embedding_matrix = np.zeros((vocab_size, config.embedding_size))
Shared_Embedding = Embedding(output_dim=config.embedding_size,
                             input_dim=vocab_size, 
                             weights=[embedding_matrix], 
                             input_length=128)

In [ ]:
# La entrada se conecta a los embeddings (de indices a embeddings) 
# word_embedding_context = Shared_Embedding(input_context)

In [ ]:
# LSTM decodificadora 
sentence_embedding_size = 128
LSTM_decoder = LSTM(sentence_embedding_size, kernel_initializer='lecun_uniform')

In [ ]:
# La entrada se conecta a los embeddings (de indices a embeddings) 
word_embedding_caption = Shared_Embedding(caption)

In [ ]:
# Decode caption embeddins
caption_embedding = LSTM_decoder(word_embedding_caption,
                                 initial_state=[input_context, input_context])

In [ ]:
# Se  conecta con una capa que representa a la palabra que se prende
out = Dense(int(vocab_size/2), activation="relu")(caption_embedding)
out = Dense(vocab_size, activation="softmax")(out)

In [ ]:
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input

img = image.load_img('10k/part-0-to-1000/y-u-no/y-u-no.jpg', target_size=(299, 299))
x = image.img_to_array(img)
print(x.shape)
x = np.expand_dims(x, axis=0)
print(x.shape)
x = preprocess_input(x)
print(x.shape)
preds = conv_model.predict(x)
preds.shape

In [ ]:
from keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(include_top=False)
img = image.load_img('10k/part-0-to-1000/y-u-no/y-u-no.jpg', target_size=(299, 299))
x = image.img_to_array(img)
print(x.shape)
x = np.expand_dims(x, axis=0)
print(x.shape)
x = preprocess_input(x)
print(x.shape)
preds = base_model.predict(x)
preds.shape